In [5]:
import pandas as pd
import numpy as np

import datetime as dt # basic functionality 
from datetime import datetime
import pandas as pd  # basic functionality
import numpy as np  # basic functionality
import mysql.connector #mySQL connector
import snowflake.connector #snowflake connector
import collections #creates product-specific data
import os
import sys #read files in specific locations
import csv #allows csv exports
from snowflake.sqlalchemy import URL #connect to protem
from sqlalchemy import create_engine #connect to protem
from pytz import timezone

snowflake_account = 'vvcvfcd-wsb62681'
snowflake_user = snowflake_test_account
snowflake_password = snowflake_test_password
snowflake_role = snowflake_test_role
snowflake_database = "FINAL_PROJECT"
snowflake_warehouse = "COMPUTE_WH"
schema = "PUBLIC"

def snowflake_reader(query):
    import snowflake.connector
    from sqlalchemy import create_engine
    from snowflake.sqlalchemy import URL
    engine = create_engine(URL(
        account = snowflake_account,
        user = snowflake_user,
        password = snowflake_password,
        database = snowflake_database,
        schema = schema,
        role = snowflake_role,
        warehouse = snowflake_warehouse
        
    ))
    connection = engine.connect()
    dataframe = pd.read_sql(query, con=engine)
    connection.close()
    engine.dispose()
    return dataframe



In [6]:
show_table = snowflake_reader('''
show tables;
''')
show_table

,created_on,name,database_name,schema_name,kind,comment,cluster_by,rows,bytes,owner,retention_time,automatic_clustering,change_tracking,is_external
0,2023-04-28 05:48:34.278000+00:00,EV_CHARGING_STATION,FINAL_PROJECT,PUBLIC,TABLE,,,660,11776,ACCOUNTADMIN,1,OFF,OFF,N
1,2023-04-28 03:36:20.614000+00:00,LDV_ZEV_SHARE_DATA,FINAL_PROJECT,PUBLIC,TABLE,,,59,4608,ACCOUNTADMIN,1,OFF,OFF,N
2,2023-04-28 03:36:57.675000+00:00,MHD_ZEV_SHARE_DATA,FINAL_PROJECT,PUBLIC,TABLE,,,237,6656,ACCOUNTADMIN,1,OFF,OFF,N
3,2023-04-28 03:39:24.405000+00:00,VEHICLE_POPULATION,FINAL_PROJECT,PUBLIC,TABLE,,,21887,156672,ACCOUNTADMIN,1,OFF,OFF,N
4,2023-04-28 03:34:50.571000+00:00,ZEV_SALE_DATA,FINAL_PROJECT,PUBLIC,TABLE,,,15591,110080,ACCOUNTADMIN,1,OFF,OFF,N


In [7]:
show_ev_charging_station_table = snowflake_reader('''
select * from EV_CHARGING_STATION;
''')
show_ev_charging_station_table

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,4,66,1473,2465,385,37,4430,2022,Q4
1,Alpine,0,0,6,0,0,0,6,2022,Q4
2,Amador,0,0,26,0,8,0,34,2022,Q4
3,Butte,6,0,59,55,26,0,146,2022,Q4
4,Calaveras,0,2,11,0,12,0,25,2022,Q4
...,...,...,...,...,...,...,...,...,...,...
655,Ventura,6,0,392,234,109,8,749,2020,Q2
656,Yolo,10,0,147,89,18,0,264,2020,Q2
657,Yuba,0,0,9,5,0,0,14,2020,Q2
658,Unknown,0,0,2,14,0,0,16,2020,Q2


In [8]:
show_ev_charging_station_table.describe()

,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total
count,660.00000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,8.60000,10.506061,894.760606,1378.666667,211.203030,18.415152,2532.942424
std,35.54126,46.325807,3561.613466,5735.209730,833.148739,74.707099,10221.283029
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.00000,0.000000,12.000000,4.000000,12.000000,0.000000,32.000000
50%,0.00000,0.000000,59.000000,31.000000,41.500000,1.000000,151.500000
75%,4.00000,1.000000,392.750000,207.000000,118.250000,9.000000,753.000000
max,335.00000,401.000000,29725.000000,49920.000000,8533.000000,674.000000,88267.000000


## EV charger file

In [9]:
ev_charger_Q4_22 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q4 2022")
ev_charger_Q4_22 = ev_charger_Q4_22.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q4_22['county'] = ev_charger_Q4_22['county'].astype('str')
ev_charger_Q4_22['year'] = '2022'
ev_charger_Q4_22['year'] = ev_charger_Q4_22['year'].astype('str')
ev_charger_Q4_22['quarter'] = 'Q4'
ev_charger_Q4_22['year'] = ev_charger_Q4_22['year'].astype('str')
ev_charger_Q4_22['public_level_1'] = ev_charger_Q4_22['public_level_1'].astype('int64')
ev_charger_Q4_22['shared_private_level_1'] = ev_charger_Q4_22['shared_private_level_1'].astype('int64')
ev_charger_Q4_22['public_level_2'] = ev_charger_Q4_22['public_level_2'].astype('int64')
ev_charger_Q4_22['shared_private_level_2'] = ev_charger_Q4_22['shared_private_level_2'].astype('int64')
ev_charger_Q4_22['public_dc_fast'] = ev_charger_Q4_22['public_dc_fast'].astype('int64')
ev_charger_Q4_22['shared_private_dc_fast'] = ev_charger_Q4_22['shared_private_dc_fast'].astype('int64')
ev_charger_Q4_22['total'] = ev_charger_Q4_22['total'].astype('int64')
ev_charger_Q4_22.shape

(60, 10)

In [10]:
ev_charger_Q3_22 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q3 2022")
ev_charger_Q3_22 = ev_charger_Q3_22.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q3_22['county'] = ev_charger_Q3_22['county'].astype('str')
ev_charger_Q3_22['year'] = '2022'
ev_charger_Q3_22['year'] = ev_charger_Q3_22['year'].astype('str')
ev_charger_Q3_22['quarter'] = 'Q3'
ev_charger_Q3_22['year'] = ev_charger_Q3_22['year'].astype('str')
ev_charger_Q3_22['public_level_1'] = ev_charger_Q3_22['public_level_1'].astype('int64')
ev_charger_Q3_22['shared_private_level_1'] = ev_charger_Q3_22['shared_private_level_1'].astype('int64')
ev_charger_Q3_22['public_level_2'] = ev_charger_Q3_22['public_level_2'].astype('int64')
ev_charger_Q3_22['shared_private_level_2'] = ev_charger_Q3_22['shared_private_level_2'].astype('int64')
ev_charger_Q3_22['public_dc_fast'] = ev_charger_Q3_22['public_dc_fast'].astype('int64')
ev_charger_Q3_22['shared_private_dc_fast'] = ev_charger_Q3_22['shared_private_dc_fast'].astype('int64')
ev_charger_Q3_22['total'] = ev_charger_Q3_22['total'].astype('int64')
ev_charger_Q3_22.shape

(60, 10)

In [11]:
ev_charger_Q2_22 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q2 2022")
ev_charger_Q2_22 = ev_charger_Q2_22.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q2_22['county'] = ev_charger_Q2_22['county'].astype('str')
ev_charger_Q2_22['year'] = '2022'
ev_charger_Q2_22['year'] = ev_charger_Q2_22['year'].astype('str')
ev_charger_Q2_22['quarter'] = 'Q2'
ev_charger_Q2_22['year'] = ev_charger_Q2_22['year'].astype('str')
ev_charger_Q2_22['public_level_1'] = ev_charger_Q2_22['public_level_1'].astype('int64')
ev_charger_Q2_22['shared_private_level_1'] = ev_charger_Q2_22['shared_private_level_1'].astype('int64')
ev_charger_Q2_22['public_level_2'] = ev_charger_Q2_22['public_level_2'].astype('int64')
ev_charger_Q2_22['shared_private_level_2'] = ev_charger_Q2_22['shared_private_level_2'].astype('int64')
ev_charger_Q2_22['public_dc_fast'] = ev_charger_Q2_22['public_dc_fast'].astype('int64')
ev_charger_Q2_22['shared_private_dc_fast'] = ev_charger_Q2_22['shared_private_dc_fast'].astype('int64')
ev_charger_Q2_22['total'] = ev_charger_Q2_22['total'].astype('int64')
ev_charger_Q2_22.head(10)


,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,0,70,1724,2104,366,34,5139,2022,Q2
1,Alpine,0,0,16,0,0,0,16,2022,Q2
2,Amador,0,0,25,0,8,0,33,2022,Q2
3,Butte,6,0,66,46,27,1,158,2022,Q2
4,Calaveras,0,2,9,0,12,0,23,2022,Q2
5,Colusa,0,0,18,0,18,0,36,2022,Q2
6,Contra Costa,2,2,575,445,245,1,1699,2022,Q2
7,Del Norte,0,0,12,1,17,2,32,2022,Q2
8,El Dorado,1,0,144,0,48,0,193,2022,Q2
9,Fresno,8,1,577,381,225,9,1321,2022,Q2


In [13]:
ev_charger_Q1_22 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q1 2022")
ev_charger_Q1_22 = ev_charger_Q1_22.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q1_22['county'] = ev_charger_Q1_22['county'].astype('str')
ev_charger_Q1_22['year'] = '2022'
ev_charger_Q1_22['year'] = ev_charger_Q1_22['year'].astype('str')
ev_charger_Q1_22['quarter'] = 'Q1'
ev_charger_Q1_22['year'] = ev_charger_Q1_22['year'].astype('str')
ev_charger_Q1_22['public_level_1'] = ev_charger_Q1_22['public_level_1'].astype('int64')
ev_charger_Q1_22['shared_private_level_1'] = ev_charger_Q1_22['shared_private_level_1'].astype('int64')
ev_charger_Q1_22['public_level_2'] = ev_charger_Q1_22['public_level_2'].astype('int64')
ev_charger_Q1_22['shared_private_level_2'] = ev_charger_Q1_22['shared_private_level_2'].astype('int64')
ev_charger_Q1_22['public_dc_fast'] = ev_charger_Q1_22['public_dc_fast'].astype('int64')
ev_charger_Q1_22['shared_private_dc_fast'] = ev_charger_Q1_22['shared_private_dc_fast'].astype('int64')
ev_charger_Q1_22['total'] = ev_charger_Q1_22['total'].astype('int64')
ev_charger_Q1_22.head(10)


,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,0,70,1671,2080,347,34,5051,2022,Q1
1,Alpine,0,0,18,0,0,0,18,2022,Q1
2,Amador,0,0,25,0,8,0,33,2022,Q1
3,Butte,6,0,41,45,27,1,132,2022,Q1
4,Calaveras,0,2,7,0,12,0,21,2022,Q1
5,Colusa,0,0,18,0,18,0,36,2022,Q1
6,Contra Costa,2,2,589,442,252,1,1715,2022,Q1
7,Del Norte,0,0,12,1,17,2,32,2022,Q1
8,El Dorado,1,0,124,0,48,0,173,2022,Q1
9,Fresno,8,1,592,373,208,9,1311,2022,Q1


In [14]:
ev_charger_Q4_21 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q4 2021")
ev_charger_Q4_21 = ev_charger_Q4_21.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q4_21['county'] = ev_charger_Q4_21['county'].astype('str')
ev_charger_Q4_21['year'] = '2021'
ev_charger_Q4_21['year'] = ev_charger_Q4_21['year'].astype('str')
ev_charger_Q4_21['quarter'] = 'Q4'
ev_charger_Q4_21['year'] = ev_charger_Q4_21['year'].astype('str')
ev_charger_Q4_21['public_level_1'] = ev_charger_Q4_21['public_level_1'].astype('int64')
ev_charger_Q4_21['shared_private_level_1'] = ev_charger_Q4_21['shared_private_level_1'].astype('int64')
ev_charger_Q4_21['public_level_2'] = ev_charger_Q4_21['public_level_2'].astype('int64')
ev_charger_Q4_21['shared_private_level_2'] = ev_charger_Q4_21['shared_private_level_2'].astype('int64')
ev_charger_Q4_21['public_dc_fast'] = ev_charger_Q4_21['public_dc_fast'].astype('int64')
ev_charger_Q4_21['shared_private_dc_fast'] = ev_charger_Q4_21['shared_private_dc_fast'].astype('int64')
ev_charger_Q4_21['total'] = ev_charger_Q4_21['total'].astype('int64')
ev_charger_Q4_21.shape
ev_charger_Q4_21.head(5)

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,6,70,1671,1583,332,35,3697,2021,Q4
1,Alpine,0,0,14,0,0,0,14,2021,Q4
2,Amador,0,0,25,6,8,0,39,2021,Q4
3,Butte,6,0,41,23,24,0,94,2021,Q4
4,Calaveras,0,2,7,0,12,0,21,2021,Q4


In [15]:
ev_charger_Q3_21 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q3 2021")
ev_charger_Q3_21 = ev_charger_Q3_21.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q3_21['county'] = ev_charger_Q3_21['county'].astype('str')
ev_charger_Q3_21['year'] = '2021'
ev_charger_Q3_21['year'] = ev_charger_Q3_21['year'].astype('str')
ev_charger_Q3_21['quarter'] = 'Q3'
ev_charger_Q3_21['year'] = ev_charger_Q3_21['year'].astype('str')
ev_charger_Q3_21['public_level_1'] = ev_charger_Q3_21['public_level_1'].astype('int64')
ev_charger_Q3_21['shared_private_level_1'] = ev_charger_Q3_21['shared_private_level_1'].astype('int64')
ev_charger_Q3_21['public_level_2'] = ev_charger_Q3_21['public_level_2'].astype('int64')
ev_charger_Q3_21['shared_private_level_2'] = ev_charger_Q3_21['shared_private_level_2'].astype('int64')
ev_charger_Q3_21['public_dc_fast'] = ev_charger_Q3_21['public_dc_fast'].astype('int64')
ev_charger_Q3_21['shared_private_dc_fast'] = ev_charger_Q3_21['shared_private_dc_fast'].astype('int64')
ev_charger_Q3_21['total'] = ev_charger_Q3_21['total'].astype('int64')
ev_charger_Q3_21.shape

(60, 10)

In [16]:
ev_charger_Q2_21 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q2 2021")
ev_charger_Q2_21 = ev_charger_Q2_21.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q2_21['county'] = ev_charger_Q2_21['county'].astype('str')
ev_charger_Q2_21['year'] = '2021'
ev_charger_Q2_21['year'] = ev_charger_Q2_21['year'].astype('str')
ev_charger_Q2_21['quarter'] = 'Q2'
ev_charger_Q2_21['year'] = ev_charger_Q2_21['year'].astype('str')
ev_charger_Q2_21['public_level_1'] = ev_charger_Q2_21['public_level_1'].astype('int64')
ev_charger_Q2_21['shared_private_level_1'] = ev_charger_Q2_21['shared_private_level_1'].astype('int64')
ev_charger_Q2_21['public_level_2'] = ev_charger_Q2_21['public_level_2'].astype('int64')
ev_charger_Q2_21['shared_private_level_2'] = ev_charger_Q2_21['shared_private_level_2'].astype('int64')
ev_charger_Q2_21['public_dc_fast'] = ev_charger_Q2_21['public_dc_fast'].astype('int64')
ev_charger_Q2_21['shared_private_dc_fast'] = ev_charger_Q2_21['shared_private_dc_fast'].astype('int64')
ev_charger_Q2_21['total'] = ev_charger_Q2_21['total'].astype('int64')
ev_charger_Q2_21.shape
ev_charger_Q2_21.head(5)


,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,14,63,1633,1497,310,43,3560,2021,Q2
1,Alpine,0,0,14,1,0,0,15,2021,Q2
2,Amador,0,0,25,7,8,0,40,2021,Q2
3,Butte,6,0,44,22,17,2,91,2021,Q2
4,Calaveras,0,0,7,0,12,0,19,2021,Q2


In [17]:
ev_charger_Q1_21 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q1 2021")
ev_charger_Q1_21 = ev_charger_Q1_21.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q1_21['county'] = ev_charger_Q1_21['county'].astype('str')
ev_charger_Q1_21['year'] = '2021'
ev_charger_Q1_21['year'] = ev_charger_Q1_21['year'].astype('str')
ev_charger_Q1_21['quarter'] = 'Q1'
ev_charger_Q1_21['year'] = ev_charger_Q1_21['year'].astype('str')
ev_charger_Q1_21['public_level_1'] = ev_charger_Q1_21['public_level_1'].astype('int64')
ev_charger_Q1_21['shared_private_level_1'] = ev_charger_Q1_21['shared_private_level_1'].astype('int64')
ev_charger_Q1_21['public_level_2'] = ev_charger_Q1_21['public_level_2'].astype('int64')
ev_charger_Q1_21['shared_private_level_2'] = ev_charger_Q1_21['shared_private_level_2'].astype('int64')
ev_charger_Q1_21['public_dc_fast'] = ev_charger_Q1_21['public_dc_fast'].astype('int64')
ev_charger_Q1_21['shared_private_dc_fast'] = ev_charger_Q1_21['shared_private_dc_fast'].astype('int64')
ev_charger_Q1_21['total'] = ev_charger_Q1_21['total'].astype('int64')
ev_charger_Q1_21.shape
ev_charger_Q1_21.head(5)

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,14,63,1635,1497,292,43,3544,2021,Q1
1,Alpine,0,0,14,1,0,0,15,2021,Q1
2,Amador,0,0,25,7,8,0,40,2021,Q1
3,Butte,6,0,36,22,16,2,82,2021,Q1
4,Calaveras,0,0,7,0,12,0,19,2021,Q1


In [18]:
ev_charger_Q4_20 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q4 2020")
ev_charger_Q4_20 = ev_charger_Q4_20.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q4_20['county'] = ev_charger_Q4_20['county'].astype('str')
ev_charger_Q4_20['year'] = '2020'
ev_charger_Q4_20['year'] = ev_charger_Q4_20['year'].astype('str')
ev_charger_Q4_20['quarter'] = 'Q4'
ev_charger_Q4_20['year'] = ev_charger_Q4_20['year'].astype('str')
ev_charger_Q4_20['public_level_1'] = ev_charger_Q4_20['public_level_1'].astype('int64')
ev_charger_Q4_20['shared_private_level_1'] = ev_charger_Q4_20['shared_private_level_1'].astype('int64')
ev_charger_Q4_20['public_level_2'] = ev_charger_Q4_20['public_level_2'].astype('int64')
ev_charger_Q4_20['shared_private_level_2'] = ev_charger_Q4_20['shared_private_level_2'].astype('int64')
ev_charger_Q4_20['public_dc_fast'] = ev_charger_Q4_20['public_dc_fast'].astype('int64')
ev_charger_Q4_20['shared_private_dc_fast'] = ev_charger_Q4_20['shared_private_dc_fast'].astype('int64')
ev_charger_Q4_20['total'] = ev_charger_Q4_20['total'].astype('int64')
ev_charger_Q4_20.shape
ev_charger_Q4_20.head(5)

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,14,14,1519,1472,296,38,3353,2020,Q4
1,Alpine,0,0,14,1,0,0,15,2020,Q4
2,Amador,0,0,25,7,8,0,40,2020,Q4
3,Butte,6,0,30,22,16,2,76,2020,Q4
4,Calaveras,0,0,7,0,0,0,7,2020,Q4


In [19]:
ev_charger_Q3_20 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q3 2020")
ev_charger_Q3_20 = ev_charger_Q3_20.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DC Fast': 'public_dc_fast',
                                            'Shared Private DC Fast': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q3_20['county'] = ev_charger_Q3_20['county'].astype('str')
ev_charger_Q3_20['year'] = '2020'
ev_charger_Q3_20['year'] = ev_charger_Q3_20['year'].astype('str')
ev_charger_Q3_20['quarter'] = 'Q3'
ev_charger_Q3_20['year'] = ev_charger_Q3_20['year'].astype('str')
ev_charger_Q3_20['public_level_1'] = ev_charger_Q3_20['public_level_1'].astype('int64')
ev_charger_Q3_20['shared_private_level_1'] = ev_charger_Q3_20['shared_private_level_1'].astype('int64')
ev_charger_Q3_20['public_level_2'] = ev_charger_Q3_20['public_level_2'].astype('int64')
ev_charger_Q3_20['shared_private_level_2'] = ev_charger_Q3_20['shared_private_level_2'].astype('int64')
ev_charger_Q3_20['public_dc_fast'] = ev_charger_Q3_20['public_dc_fast'].astype('int64')
ev_charger_Q3_20['shared_private_dc_fast'] = ev_charger_Q3_20['shared_private_dc_fast'].astype('int64')
ev_charger_Q3_20['total'] = ev_charger_Q3_20['total'].astype('int64')
ev_charger_Q3_20.shape
ev_charger_Q3_20.head(5)

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,14,14,1288,1460,268,38,3082,2020,Q3
1,Alpine,0,0,12,1,0,0,13,2020,Q3
2,Amador,0,0,23,7,0,0,30,2020,Q3
3,Butte,6,0,32,20,16,2,76,2020,Q3
4,Calaveras,0,0,7,0,0,0,7,2020,Q3


In [20]:
ev_charger_Q2_20 = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/EV_Chargers_Last_updated_03-20-2023.xlsx', 
                           engine='openpyxl', sheet_name="Q2 2020")
ev_charger_Q2_20 = ev_charger_Q2_20.rename(columns=
                                           {'County': 'county',
                                            'Public Level 1': 'public_level_1',
                                            'Shared Private Level 1': 'shared_private_level_1',
                                            'Public Level 2': 'public_level_2',
                                            'Shared Private Level 2': 'shared_private_level_2',
                                            'Public DCFC': 'public_dc_fast',
                                            'Shared Private DCFC': 'shared_private_dc_fast',
                                            'Total': 'total'                                          
                                           })
ev_charger_Q2_20['county'] = ev_charger_Q2_20['county'].astype('str')
ev_charger_Q2_20['year'] = '2020'
ev_charger_Q2_20['year'] = ev_charger_Q2_20['year'].astype('str')
ev_charger_Q2_20['quarter'] = 'Q2'
ev_charger_Q2_20['year'] = ev_charger_Q2_20['year'].astype('str')
ev_charger_Q2_20['public_level_1'] = ev_charger_Q2_20['public_level_1'].astype('int64')
ev_charger_Q2_20['shared_private_level_1'] = ev_charger_Q2_20['shared_private_level_1'].astype('int64')
ev_charger_Q2_20['public_level_2'] = ev_charger_Q2_20['public_level_2'].astype('int64')
ev_charger_Q2_20['shared_private_level_2'] = ev_charger_Q2_20['shared_private_level_2'].astype('int64')
# ev_charger_Q2_20['public_dc_fast'] = ev_charger_Q2_20['public_dc_fast'].astype('int64')
ev_charger_Q2_20['shared_private_dc_fast'] = ev_charger_Q2_20['shared_private_dc_fast'].astype('int64')
ev_charger_Q2_20['total'] = ev_charger_Q2_20['total'].astype('int64')
ev_charger_Q2_20.shape
ev_charger_Q2_20.head(5)

,county,public_level_1,shared_private_level_1,public_level_2,shared_private_level_2,public_dc_fast,shared_private_dc_fast,total,year,quarter
0,Alameda,14,14,1274,1398,258,32,2990,2020,Q2
1,Alpine,0,0,12,0,0,0,12,2020,Q2
2,Amador,0,0,23,6,0,0,29,2020,Q2
3,Butte,6,0,32,20,16,0,74,2020,Q2
4,Calaveras,0,0,7,0,0,0,7,2020,Q2


In [21]:
EV_chargers = pd.concat([ev_charger_Q4_22,ev_charger_Q3_22,ev_charger_Q2_22,ev_charger_Q1_22,
                         ev_charger_Q4_21,ev_charger_Q3_21,ev_charger_Q2_21,ev_charger_Q1_21,
                         ev_charger_Q4_20,ev_charger_Q3_20,ev_charger_Q2_20])
EV_chargers.shape

(660, 10)

In [22]:
EV_chargers = EV_chargers.drop_duplicates()
EV_chargers.shape

(660, 10)

In [16]:
EV_chargers.to_csv('/Users/minhngo/Documents/GitHub/DATA225/processed_data/EV_Chargers.csv', 
                       index=False)

In [23]:
engine = create_engine(URL(
        account = snowflake_account,
        user = snowflake_user,
        password = snowflake_password,
        database = snowflake_database,
        schema = "PUBLIC",
        role = snowflake_role,
        warehouse = snowflake_warehouse
        
    ))
connection = engine.connect()
connection.execute(''' USE SCHEMA "PUBLIC" ''')

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_40596/1319413564.py:12: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  connection.execute(''' USE SCHEMA "PUBLIC" ''')


In [24]:
EV_chargers.to_sql(name='EV_CHARGING_STATION_LIVE_DEMO_NOW',
            con=engine,
            if_exists="append",
            index=False, method='multi', chunksize = 5000)

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_40596/9874505.py:1: UserWarning: The provided table name 'EV_CHARGING_STATION_LIVE_DEMO_NOW' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  EV_chargers.to_sql(name='EV_CHARGING_STATION_LIVE_DEMO_NOW',


660

In [25]:
connection.close()
engine.dispose()

## Vehicle population

In [44]:
vehicle_population = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/Vehicle_Population_Last_updated_04-29-2022.xlsx', 
                           engine='openpyxl', sheet_name="County")

vehicle_population = vehicle_population.rename(columns=
                                           {'County': 'county',
                                            'Fuel Type': 'fuel_type',
                                            'Make': 'make',
                                            'Model': 'model',
                                            'Number of Vehicles': 'unit_sold',
                                            'Data Year': 'year'                                          
                                           })

vehicle_population['county'] = vehicle_population['county'].astype('str')
vehicle_population['year'] = vehicle_population['year'].astype('str')
vehicle_population['fuel_type'] = vehicle_population['fuel_type'].astype('str')
vehicle_population['make'] = vehicle_population['make'].fillna('').astype('str')
vehicle_population['model'] = vehicle_population['model'].fillna('').astype('str')
vehicle_population['unit_sold'] = vehicle_population['unit_sold'].astype('int64')
vehicle_population.head(10)

,year,county,fuel_type,make,model,unit_sold
0,2010,Alameda,Diesel,,,10939
1,2010,Alameda,Electric,Ford,Ranger,3
2,2010,Alameda,Electric,Tesla,Roadster,17
3,2010,Alameda,Flex Fuel,,,10974
4,2010,Alameda,Gasoline,,,840577
5,2010,Alameda,Gasoline Hybrid,,,22720
6,2010,Alameda,Natural Gas,,,173
7,2010,Alameda,Propane,,,19
8,2010,Alpine,Diesel,,,73
9,2010,Alpine,Flex Fuel,,,16


In [5]:
vehicle_population.to_csv('/Users/minhngo/Documents/GitHub/DATA225/processed_data/vehicle_population.csv', 
                       index=False)

In [45]:
vehicle_population.columns = map(str.upper, vehicle_population.columns)
vehicle_population.to_sql(name='VEHICLE_POPULATION',
            con=engine,
            if_exists="append",
            index=False, method='multi', chunksize = 5000)

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_10755/771887815.py:2: UserWarning: The provided table name 'VEHICLE_POPULATION' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  vehicle_population.to_sql(name='VEHICLE_POPULATION',


21887

## ZEV (Zero emission vehicle sold)

In [36]:
zev_sales_data = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/New_ZEV_Sales_Last_updated_01-18-2023.xlsx', 
                           engine='openpyxl', sheet_name="County")
zev_sales_data = zev_sales_data.rename(columns=
                                           {'County': 'county',
                                            'FUEL_TYPE': 'fuel_type',
                                            'MAKE': 'make',
                                            'MODEL': 'model',
                                            'Number of Vehicles': 'unit_sold',
                                            'Data Year': 'year'                                          
                                           })

zev_sales_data['county'] = zev_sales_data['county'].astype('str')
zev_sales_data['year'] = zev_sales_data['year'].astype('str')
zev_sales_data['fuel_type'] = zev_sales_data['fuel_type'].astype('str')
zev_sales_data['make'] = zev_sales_data['make'].fillna('').astype('str')
zev_sales_data['model'] = zev_sales_data['model'].fillna('').astype('str')
zev_sales_data['unit_sold'] = zev_sales_data['unit_sold'].astype('int64')
zev_sales_data.head(10)

,year,county,fuel_type,make,model,unit_sold
0,1998,Los Angeles,Electric,Ford,Ranger,1
1,1998,Orange,Electric,Ford,Ranger,1
2,1998,San Bernardino,Electric,Ford,Ranger,2
3,1998,San Mateo,Electric,Ford,Ranger,1
4,1999,Santa Barbara,Electric,Ford,Ranger,1
5,2000,Out Of State,Electric,Ford,Ranger,2
6,2001,Los Angeles,Electric,Ford,Ranger,1
7,2002,Tulare,Electric,Ford,Ranger,1
8,2002,Ventura,Electric,Ford,Ranger,2
9,2003,Placer,Electric,Ford,Ranger,2


In [10]:
zev_sales_data.to_csv('/Users/minhngo/Documents/GitHub/DATA225/processed_data/zev_sales_data.csv', 
                       index=False)

In [38]:
zev_sales_data.columns = map(str.upper, zev_sales_data.columns)
zev_sales_data.to_sql(name='ZEV_SALE_DATA',
            con=engine,
            if_exists="append",
            index=False, method='multi', chunksize = 5000)

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_10755/339982266.py:1: UserWarning: The provided table name 'ZEV_SALE_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  zev_sales_data.to_sql(name='ZEV_SALE_DATA',


15591

## ZEV share per county

In [39]:
ldv_zev_share = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/LDV_Sales_and_Shares_Last_updated_01-18-2023.xlsx', 
                           engine='openpyxl', sheet_name="County")
ldv_zev_share = ldv_zev_share.rename(columns=
                                           {'County': 'county',
                                            'ZEV Sales Share \n(2022 Q1)': 'zev_sale_share_Q1_2022',
                                            'ZEV Sales Share \n(2022 Q1-Q2)': 'zev_sale_share_Q1_Q2_2022',
                                            'ZEV Sales Share \n(2022 Q1-Q3)': 'zev_sale_share_Q1_Q3_2022',
                                            'ZEV Sales Share \n(2022 Q1-Q4)': 'zev_sale_share_Q1_Q4_2022'                                         
                                           })
ldv_zev_share['county'] = ldv_zev_share['county'].astype('str')
ldv_zev_share['zev_sale_share_Q1_2022'] = ldv_zev_share['zev_sale_share_Q1_2022'].astype('float64')
ldv_zev_share['zev_sale_share_Q1_Q2_2022'] = ldv_zev_share['zev_sale_share_Q1_Q2_2022'].astype('float64')
ldv_zev_share['zev_sale_share_Q1_Q3_2022'] = ldv_zev_share['zev_sale_share_Q1_Q3_2022'].astype('float64')
ldv_zev_share['zev_sale_share_Q1_Q4_2022'] = ldv_zev_share['zev_sale_share_Q1_Q4_2022'].astype('float64')
ldv_zev_share = ldv_zev_share[['county','zev_sale_share_Q1_2022','zev_sale_share_Q1_Q2_2022',
                                 'zev_sale_share_Q1_Q3_2022','zev_sale_share_Q1_Q4_2022']]
ldv_zev_share.head(10)

,county,zev_sale_share_Q1_2022,zev_sale_share_Q1_Q2_2022,zev_sale_share_Q1_Q3_2022,zev_sale_share_Q1_Q4_2022
0,Alameda,0.288839,0.286781,0.298400,0.311728
1,Alpine,0.083333,0.107143,0.116279,0.163265
2,Amador,0.070740,0.077586,0.088993,0.085532
3,Butte,0.068717,0.069885,0.078024,0.083242
4,Calaveras,0.089362,0.091723,0.085851,0.084211
5,Colusa,0.012195,0.040580,0.040650,0.043155
6,Contra Costa,0.224155,0.226816,0.242483,0.260150
7,Del Norte,0.030769,0.032000,0.027701,0.029979
8,El Dorado,0.141905,0.155495,0.164726,0.176353
9,Fresno,0.089097,0.096784,0.100041,0.105282


In [23]:
ldv_zev_share.to_csv('/Users/minhngo/Documents/GitHub/DATA225/processed_data/ldv_zev_share.csv', 
                       index=False)

In [ ]:
Index(['County', 'Total LDV Sales \n(2022 Q1)',
       'Total LDV Sales \n(2022 Q1-Q2)', 'Total LDV Sales \n(2022 Q1-Q3)',
       'Total LDV Sales \n(2022 Q1-Q4)', 'ZEV Sales Share \n(2022 Q1)',
       'ZEV Sales Share \n(2022 Q1-Q2)', 'ZEV Sales Share \n(2022 Q1-Q3)',
       'ZEV Sales Share \n(2022 Q1-Q4)'],
      dtype='object')

In [40]:
ldv_zev_share.columns = map(str.upper, ldv_zev_share.columns)
ldv_zev_share.to_sql(name='LDV_ZEV_SHARE_DATA',
            con=engine,
            if_exists="append",
            index=False, method='multi', chunksize = 5000)

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_10755/777458337.py:2: UserWarning: The provided table name 'LDV_ZEV_SHARE_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  ldv_zev_share.to_sql(name='LDV_ZEV_SHARE_DATA',


59

## Medium heavy duty vehicle population

In [41]:
mhd_zev_sales_data = pd.read_excel('/Users/minhngo/Documents/GitHub/DATA225/raw_data/Medium_Heavy_Duty_Vehicle_Population_Last_updated_07-26-2022.xlsx', 
                           engine='openpyxl', sheet_name="MDHD Vehicle Population")
mhd_zev_sales_data = mhd_zev_sales_data.rename(columns=
                                           {'County': 'county',
                                            'Fuel': 'fuel_type',
                                            'MakeName': 'make',
                                            'Model': 'model',
                                            'Class': 'class',
                                            'BodyStyle': 'body_style',
                                            'VehicleType': 'vehicle_type',
                                            'Number of Vehicles': 'unit_sold'                                       
                                           })
mhd_zev_sales_data = mhd_zev_sales_data[['county','fuel_type','make',
                                 'model','body_style','vehicle_type','unit_sold']]

mhd_zev_sales_data['county'] = mhd_zev_sales_data['county'].astype('str')
mhd_zev_sales_data['fuel_type'] = mhd_zev_sales_data['fuel_type'].astype('str')
mhd_zev_sales_data['make'] = mhd_zev_sales_data['make'].astype('str')
mhd_zev_sales_data['body_style'] = mhd_zev_sales_data['body_style'].fillna('').astype('str')
mhd_zev_sales_data['vehicle_type'] = mhd_zev_sales_data['vehicle_type'].fillna('').astype('str')
mhd_zev_sales_data['model'] = mhd_zev_sales_data['model'].fillna('').astype('str')
mhd_zev_sales_data['unit_sold'] = mhd_zev_sales_data['unit_sold'].astype('int64')

mhd_zev_sales_data.head(10)

,county,fuel_type,make,model,body_style,vehicle_type,unit_sold
0,Alameda,Electric,Blue Bird,All American / All Canadian,School Bus,Bus,18
1,Alameda,Electric,BYD Coach And Bus,Electric Truck,Onroad Drayage Tractor,Truck,8
2,Alameda,Electric,BYD Coach And Bus,Transit Bus 40F,Transit Bus,Bus,1
3,Alameda,Electric,Greenpower Motor,EVC210,Transit Bus,Bus,1
4,Alameda,Electric,Lion Ebus,Lion C V2,School Bus,Bus,2
5,Alameda,Electric,Smith Electric,D100 Sc,Straight Truck,Truck,1
6,Alameda,Electric,Smith Electric,D75 Sc,Step-Van,Van,1
7,Alameda,Fuel Cell,New Flyer,Xcelsior,Transit Bus,Bus,29
8,Alpine,Electric,Blue Bird,All American / All Canadian,School Bus,Bus,1
9,Amador,Electric,Lion Ebus,Lion C V2,School Bus,Bus,1


In [42]:
mhd_zev_sales_data.columns = map(str.upper, mhd_zev_sales_data.columns)
mhd_zev_sales_data.to_sql(name='MHD_ZEV_SHARE_DATA',
            con=engine,
            if_exists="append",
            index=False, method='multi', chunksize = 5000)

/var/folders/9v/dm04gj693l122lr8pvxh6h3m0000gn/T/ipykernel_10755/2693420251.py:2: UserWarning: The provided table name 'MHD_ZEV_SHARE_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  mhd_zev_sales_data.to_sql(name='MHD_ZEV_SHARE_DATA',


237